In [10]:
import random
import os
import json
import pickle       # for serialization
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer     #verify the same word
# connect database
from db import train_table

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Dropout
from tensorflow.python.keras.optimizers import gradient_descent_v2

In [11]:
lemmatizer = WordNetLemmatizer()        #verify the same word

intents = json.loads(open('.././data/intents.json').read())       #Load file json

words = []          # All the words
classes = []        # All class 
documents = []
ignore_letters = ['?', '!', '.', ',']       # all ignore letters

In [12]:
for intent in intents["intents"]:
        for pattern in intent['patterns']:
            word_list = nltk.word_tokenize(pattern)  # it look like split function ( 'what your name' => ['what', 'your','name'] )
            words.extend(word_list)     # add to array words
            documents.append((word_list, intent['tag']))   # add all word with tag to array documents
            if intent['tag'] not in classes:
                classes.append(intent['tag'])       # check if tag already have in classes and then add to array classes
print(classes)

['greetings', 'farewells', 'age', 'gender', 'occupation', 'web development languages', 'cheer up', 'request_assistance', 'start', 'origin', 'doubts', 'confirmation', 'name', 'thanks']


In [13]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [14]:


classes = sorted(set(classes))

pickle.dump(words, open('.././data/words.pkl', 'wb'))
pickle.dump(classes, open('.././data/classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)


In [15]:
print("documents" + str(documents[:10]))

documents[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['Greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going', '?'], 'greetings'), (['how', 'are', 'you'], 'greetings'), (['Hey', 'there', '!'], 'greetings'), (['Morning', '!'], 'greetings')]


In [16]:
i =0
for document in documents:
        i+=1
        bag = []
        word_patterns = document[0]
        word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
        # print(str(word_patterns) + '\n')
        for word in words:
            bag.append(1) if word in word_patterns else bag.append(0)
        # print(bag)

        output_row = list(output_empty)
        output_row[classes.index(document[1])] = 1
        training.append([bag, output_row])
        # print(output_row)
        if i == 10: break 

In [17]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\phanp\AppData\Local\Temp\ipykernel_30376\4024136173.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [18]:
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))     # limit overfitting
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))     # limit overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))     # limit overfitting
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = gradient_descent_v2.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fit and save history
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=10, verbose=1)
model.save('./data/chatbot_model.h5', hist)

Epoch 1/200
1/1 [==============================] - 0s 328ms/step - loss: 2.7370 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 4ms/step - loss: 2.6787 - accuracy: 0.1000
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 2.7269 - accuracy: 0.1000
Epoch 4/200
1/1 [==============================] - 0s 4ms/step - loss: 2.4837 - accuracy: 0.5000
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 2.4577 - accuracy: 0.5000
Epoch 6/200
1/1 [==============================] - 0s 4ms/step - loss: 2.4257 - accuracy: 0.7000
Epoch 7/200
1/1 [==============================] - 0s 4ms/step - loss: 2.3444 - accuracy: 0.8000
Epoch 8/200
1/1 [==============================] - 0s 3ms/step - loss: 2.2372 - accuracy: 0.9000
Epoch 9/200
1/1 [==============================] - 0s 3ms/step - loss: 2.0619 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 3ms/step - loss: 1.8994 - accuracy: 1.0000
Epoch 11/200
1/1 [=====